In [3]:
import requests
from flask import Flask, render_template, request, jsonify
import json
from datetime import datetime
import time
import folium


app = Flask(__name__)
CANVAS_API_URL = 'Your-URL'
CANVAS_API_KEY = 'Your-API-KEY'

def make_canvas_api_request(url, params=None):
    headers = {'Authorization': 'Bearer ' + CANVAS_API_KEY}
    response = requests.get(CANVAS_API_URL + url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()
@app.route('/')
def index():
    return render_template('Login.html')
@app.route('/assistant')
def assist():
    return render_template('Remix_final.html')
@app.route('/schedule')
def schedule():
    return render_template('Scheduler.html')
@app.route('/faq')
def faq():
    return render_template('FAQ.html')


@app.route('/profile', methods=['POST'])
def profile():
    name = request.form.get('uname')
    name = str(name).replace('@clarku.edu','')
    return render_template('Profile.html',name = name)
@app.route('/login')
def login():
    return render_template('Login.html')
@app.route('/maps')
def maps():
    # Create a map object centered at a particular location
    latitude, longitude = [ 42.24706, -71.8285614]
    map = folium.Map(location=[latitude, longitude], zoom_start=12)
    current_marker = folium.Marker(location=[latitude, longitude], icon=folium.Icon(color='red'))
    # Create markers for three locations
    
    marker1 = folium.Marker(location=[42.251881, -71.824430], popup="Algorithms Class\n Time: 7-8 PM")
    marker2 = folium.Marker(location=[42.250762, -71.822291], popup="History Class\n Time: 5-8 PM")
    marker3 = folium.Marker(location=[42.250468, -71.823997], popup="Economics Class\n Time: 1-3 PM")
    marker4 = folium.Marker(location=[42.252771, -71.823048], popup="Event\n Time: 1-3 PM")

    # Add markers to the map
    current_marker.add_to(map)
    marker1.add_to(map)
    marker2.add_to(map)
    marker3.add_to(map)
    marker4.add_to(map)

    # Render the template with the map
    return render_template('maps.html', map=map._repr_html_())

@app.route('/location')
def location():
    if 'geolocation' in request.headers:
        # Parse the geolocation data from the headers
        geolocation = request.headers['geolocation']
        latitude, longitude = geolocation.split(',')
        return jsonify({'latitude': latitude, 'longitude': longitude})
    else:
        return jsonify({'error': 'Geolocation data not available'})


@app.route('/chatbot', methods=['POST'])
def chatbot():
    num = {1:'First',2: 'Second',3:'Third',4:'Fourth',5:'Fifth'}
    # Get the message from the client
    message = request.json['message']
    response1 = "Sorry, Didn't get that"
   
    if 'courses' in message:
        courses = make_canvas_api_request('/courses')
        response1 = 'Here are your Canvas courses:\n\n'
        for course in courses:
            response1 += f'{course["name"]}\n' +'<br>'
    elif 'to do' in message or 'to-do' in message:
        results = make_canvas_api_request('/users/self/todo')
        items = []
        #print(results)
        for result in results:
            title = result["assignment"]["name"]
            due_date = result["assignment"]["due_at"]
            c_name = result["context_name"]
            items.append((title, due_date,c_name))
         # Format the to-do list as a string
        response1 = 'Here are your To do list items \n\n<br>'
        response1 += "There are a total of :" +str(len(items)) + "items\n<br><br>"
        c = 1
        for i in items:
            time.sleep(2)
            response1 +=num[c] +', '+i[2]+", "+i[0].replace('_', ' ')+ " due by "+ datetime.strptime(i[1],'%Y-%m-%dT%H:%M:%SZ').strftime("%B %d") +". <br>"
            c += 1

    # Generate a response
    response = {'message': '' + response1}

    # Convert the response to speech using the Web Speech API
    utterance = {'text': response['message']}

    return jsonify(response)

if __name__ == '__main__':
    app.run()
